In [0]:
import pandas as pd
import numpy as np
import random

In [0]:
URL_ENDPOINT = "http://cs.mcgill.ca/~ksinha4/datasets/kaggle/"

train_x = np.loadtxt(URL_ENDPOINT+"train_x.csv", delimiter=",")
train_y = np.loadtxt(URL_ENDPOINT+"train_y.csv", delimiter=",")
test_x = np.loadtxt(URL_ENDPOINT+"test_x.csv", delimiter=",")

In [0]:
#neural network implemented from the following tutorial http://neuralnetworksanddeeplearning.com/chap1.html


class Network(object):

    def __init__(self, layers):
        
        self.num_layers = len(layers)
        self.layers = layers
        self.bs = [np.random.randn(y, 1) for y in layers[1:]]
        self.ws = [np.random.randn(y, x)
                        for x, y in zip(layers[:-1], layers[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.bs, self.ws):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    #stochastic gradient descent
    def SGD(self, training_data, epo, mini_batch_size, lr, reg, l2=True,
            test_data=None):
    	  # optional test data which if provided will be evaluated against.
        if test_data: n_test = len(test_data)
        
        n = len(training_data)
        for j in xrange(epo):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, lr, reg)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta, reg, l2=False):
        # initialiize empty biases
        nabla_b = [np.zeros(b.shape) for b in self.bs]
        # initialiize empty weights
        nabla_w = [np.zeros(w.shape) for w in self.ws]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        #conduct l2_regularization:
        self.ws = [(1+reg)*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.ws, nabla_w)]
        self.bs = [(1+reg)*b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.bs, nabla_b)]
          

    def backprop(self, x, y):
        #the respective arrays will contain the gradient for the cost function, and the
        #indexes will function as the layer number.
        nabla_b = [np.zeros(b.shape) for b in self.bs]
        nabla_w = [np.zeros(w.shape) for w in self.ws]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.bs, self.ws):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        #iterating the layers backwards, in otherwords closer to the 
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.ws[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        #returns the test results for neural network.
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        #returns the vector of partial derivative
        return (output_activations-y)

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [0]:
from scipy import ndimage
from skimage.transform import resize
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from google.colab import files

In [0]:
def find_largest_bounding_box(im):
  # convert to binary image
#   _, im = cv2.threshold(im,254,255,cv2.THRESH_BINARY)
  im[im<255]=0
  im[im==255]=1
  
  label_im, nb_labels = ndimage.label(im)
  
  # find largest connected component
  sizes = ndimage.sum(im, label_im, range(nb_labels + 1))
  mask_size = sizes < 4**2
  remove_pixel = mask_size[label_im]
  label_im[remove_pixel] = 0
  labels = np.unique(label_im)
  label_im = np.searchsorted(labels, label_im)

  digits = ndimage.find_objects(label_im)
  max_side = 0
  max_ind = 0
  for i in range(0,len(digits)):
    bound = im[digits[i]]
    if np.max(bound.shape) > max_side:
      max_side = np.max(bound.shape)
      max_ind = i

  bound_rec = im[digits[max_ind]]
  
  # pad and resize
  if bound_rec.shape[0] > bound_rec.shape[1]:
    resize_short = round(24*bound_rec.shape[1]/bound_rec.shape[0])
    bound_rec = resize(bound_rec, (24, resize_short), mode='reflect')
    pad_before = (np.ceil((28 - resize_short)/2)).astype(int)
    pad_after = (np.floor((28 - resize_short)/2)).astype(int)
    bound_squ = np.pad(bound_rec,((2,2),(pad_before, pad_after)),'constant', constant_values=0)
    
  else:
    resize_short = round(24*bound_rec.shape[0]/bound_rec.shape[1])
    bound_rec = resize(bound_rec, (resize_short, 24), mode='reflect')
    pad_before = (np.ceil((28 - resize_short)/2)).astype(int)
    pad_after = (np.floor((28 - resize_short)/2)).astype(int)
    bound_squ = np.pad(bound_rec,((pad_before, pad_after),(2,2)),'constant', constant_values=0)
#   _, bound_squ = cv2.threshold(bound_squ,254,255,cv2.THRESH_BINARY)
  bound_squ[bound_squ<1] = 0
  
  return bound_squ


def preprocess(train):
  prep = np.zeros((len(train),28,28))
  for i in range(0,len(prep)):
    prep[i] = find_largest_bounding_box(train[i].copy())
  return prep

In [0]:
def vectorized_result(j):
    #opposite of to_categorical
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [10]:
# reshape to original data form
x = train_x.reshape(-1, 64, 64)
y = train_y.reshape(-1, 1) 
test = test_x.reshape(-1, 64, 64)

prep_x = preprocess(x)
#np.savetxt('prep_train_x.csv',prep_x.reshape(-1,28**2),delimiter=',')
prep_test = preprocess(test)
#np.savetxt('prep_test_x.csv',prep_test.reshape(-1,28**2),delimiter=',')

/usr/local/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:431: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  safe = ((np.issubdtype(dt, int) and dt.itemsize <= int_size) or


In [0]:
#change the shape into a vector
prep_x_2 = [x.reshape(784, 1) for x in prep_x]
prep_x_2 = np.array(prep_x_2)
t_y = np.array([vectorized_result(int(i[0])) for i in y])

In [0]:
#code retreived from https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [35]:
shuffle_train, shuffle_y = shuffle_in_unison(prep_x_2, t_y)
#set network parameters here
#the number of elements in the array of Network([arr]) is the number of layers, and the corresponding number is number of neurons in that layer. In otherwords, first layer has 784 neurons
# and the last layer has 10 neurons.
net = Network([784, 16, 10])
net.SGD(zip(shuffle_train[:40000], shuffle_y[:40000]), 50, 200, 0.005, 0, test_data=zip(shuffle_train[40000:],shuffle_y[40000:]))

4
Epoch 0: 1008 / 10000
4
Epoch 1: 1048 / 10000
4
Epoch 2: 1056 / 10000
4
Epoch 3: 1081 / 10000
4
Epoch 4: 1131 / 10000
4
Epoch 5: 1153 / 10000
4
Epoch 6: 1182 / 10000
4
Epoch 7: 1205 / 10000
4
Epoch 8: 1230 / 10000
4
Epoch 9: 1261 / 10000
4
Epoch 10: 1290 / 10000
4
Epoch 11: 1306 / 10000
4
Epoch 12: 1332 / 10000
4
Epoch 13: 1343 / 10000
4
Epoch 14: 1356 / 10000
4
Epoch 15: 1369 / 10000
4
Epoch 16: 1369 / 10000
4
Epoch 17: 1378 / 10000
4
Epoch 18: 1385 / 10000
4
Epoch 19: 1398 / 10000
4
Epoch 20: 1401 / 10000
4
Epoch 21: 1391 / 10000
4
Epoch 22: 1385 / 10000
0
Epoch 23: 1382 / 10000
0
Epoch 24: 1381 / 10000
0
Epoch 25: 1382 / 10000
0
Epoch 26: 1384 / 10000
0
Epoch 27: 1386 / 10000
0
Epoch 28: 1394 / 10000
0
Epoch 29: 1406 / 10000
0
Epoch 30: 1412 / 10000
0
Epoch 31: 1415 / 10000
0
Epoch 32: 1419 / 10000
0
Epoch 33: 1419 / 10000
0
Epoch 34: 1425 / 10000
0
Epoch 35: 1428 / 10000
0
Epoch 36: 1427 / 10000
0
Epoch 37: 1426 / 10000
0
Epoch 38: 1429 / 10000
0
Epoch 39: 1432 / 10000
0
Epoch 40